In [287]:
import numpy as np
from matplotlib import pyplot as plt

In [119]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


In [120]:
y_train_valid = y_train_valid-769
y_test = y_test-769

In [184]:
X_train_valid = X_train_valid[:,:,:22].reshape(2115,1000,22)
X_test = X_test[:,:,:22].reshape(443,1000,22)

### Shape of data

In [185]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 1000, 22)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### Start

In [265]:
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Flatten,Dropout
from keras.layers import LSTM,Conv1D,MaxPooling1D,RNN,TimeDistributed,BatchNormalization,GRU
from keras.optimizers import SGD

In [280]:
model = Sequential()
conv_layer = Conv1D(filters=100,kernel_size=3,data_format="channels_last",
                 activation='relu')
max_pool = MaxPooling1D(pool_size=2,strides=2,data_format="channels_last")
model.add(Conv1D(filters=100,kernel_size=3,data_format="channels_last",
                 activation='relu',input_shape=(1000,22)))#(100,998)
model.add(conv_layer) #(100,996)
model.add(max_pool) #(100,498)
model.add(conv_layer) #100,496
model.add(conv_layer) #100,494
model.add(max_pool) #100,247
model.add(conv_layer) #100,245
model.add(conv_layer) #100,243
model.add(max_pool) #100,122
model.add(conv_layer) #100,120
model.add(conv_layer) #100,118
model.add(max_pool) #100,59
model.add(Dense(units=52,use_bias=True,activation='relu')) #100*59,4
model.add(Dropout(0.5))
model.add(Flatten(data_format="channels_last"))
model.add(Dense(units=4,use_bias=True,activation='softmax'))
print model.output_shape

(None, 4)


In [281]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [282]:
history_CNN=model.fit(X_train_valid,y_train_valid,epochs=10,batch_size=200,verbose=2,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/10
 - 40s - loss: 1.5257 - sparse_categorical_accuracy: 0.2606 - val_loss: 1.3880 - val_sparse_categorical_accuracy: 0.2217
Epoch 2/10
 - 33s - loss: 1.3867 - sparse_categorical_accuracy: 0.2585 - val_loss: 1.3875 - val_sparse_categorical_accuracy: 0.2311
Epoch 3/10
 - 29s - loss: 1.3859 - sparse_categorical_accuracy: 0.2507 - val_loss: 1.3855 - val_sparse_categorical_accuracy: 0.2594
Epoch 4/10
 - 28s - loss: 1.3859 - sparse_categorical_accuracy: 0.2622 - val_loss: 1.3861 - val_sparse_categorical_accuracy: 0.2547
Epoch 5/10
 - 26s - loss: 1.3854 - sparse_categorical_accuracy: 0.2754 - val_loss: 1.3859 - val_sparse_categorical_accuracy: 0.2500
Epoch 6/10
 - 27s - loss: 1.3847 - sparse_categorical_accuracy: 0.2780 - val_loss: 1.3860 - val_sparse_categorical_accuracy: 0.2358
Epoch 7/10
 - 25s - loss: 1.3839 - sparse_categorical_accuracy: 0.2612 - val_loss: 1.3867 - val_sparse_categorical_accuracy: 0.2217
Epoch 8/10
 - 26s - loss: 1.3

In [283]:
model_evaluate=model.evaluate(X_test,y_test)
print (model_evaluate)

443/443 [==============================] - 2s 5ms/step
[1.3818096027417324, 0.2641083517744633]


In [284]:
CNN_model=model

In [233]:
model = Sequential()
lstm = LSTM(units=4,use_bias=False,return_sequences=True)
model.add(Dropout(0.5))
model.add(LSTM(units=4,use_bias=False,return_sequences=True,input_shape=(1000,22)))
model.add(lstm)
# model.add(lstm)
# model.add(lstm)
# print model.output_shape
model.add(Dense(units=52,use_bias=True,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=4,use_bias=True,activation='softmax'))

In [234]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [235]:
history_RNN=model.fit(X_train_valid,y_train_valid,epochs=10,batch_size=100,verbose=1,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/10
1903/1903 [==============================] - 243s 128ms/step - loss: 1.3781 - sparse_categorical_accuracy: 0.3085 - acc: 0.3085 - val_loss: 1.3120 - val_sparse_categorical_accuracy: 0.3585 - val_acc: 0.3585
Epoch 2/10
1903/1903 [==============================] - 239s 126ms/step - loss: 1.2635 - sparse_categorical_accuracy: 0.4283 - acc: 0.4283 - val_loss: 1.2087 - val_sparse_categorical_accuracy: 0.4340 - val_acc: 0.4340
Epoch 3/10
1903/1903 [==============================] - 236s 124ms/step - loss: 1.1524 - sparse_categorical_accuracy: 0.4950 - acc: 0.4950 - val_loss: 1.0636 - val_sparse_categorical_accuracy: 0.5425 - val_acc: 0.5425
Epoch 4/10
1903/1903 [==============================] - 237s 125ms/step - loss: 1.0523 - sparse_categorical_accuracy: 0.5528 - acc: 0.5528 - val_loss: 0.9994 - val_sparse_categorical_accuracy: 0.5566 - val_acc: 0.5566
Epoch 5/10
1903/1903 [==============================] - 236s 124ms/step - loss: 0

In [236]:
RNN_evaluate=model.evaluate(X_test,y_test)

443/443 [==============================] - 4s 8ms/step


In [239]:
print model.metrics_names,RNN_evaluate

['loss', 'sparse_categorical_accuracy', 'acc'] [0.9881272500309395, 0.6049661415694261, 0.6049661415694261]


In [240]:
RNN_model1 = model

In [275]:
model = Sequential()
lstm = LSTM(units=10,use_bias=False,return_sequences=True)
model.add(Dropout(0.5))
model.add(LSTM(units=10,use_bias=False,return_sequences=True,input_shape=(1000,22)))
model.add(lstm)
model.add(lstm)
model.add(lstm)
# print model.output_shape
model.add(Dense(units=52,use_bias=True,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=4,use_bias=True,activation='softmax'))

In [276]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
history_RNN2=model.fit(X_train_valid,y_train_valid,epochs=10,batch_size=200,verbose=1,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/10
1903/1903 [==============================] - 56s 29ms/step - loss: 1.3869 - sparse_categorical_accuracy: 0.2433 - val_loss: 1.3862 - val_sparse_categorical_accuracy: 0.2311
Epoch 2/10
1903/1903 [==============================] - 49s 26ms/step - loss: 1.3853 - sparse_categorical_accuracy: 0.2706 - val_loss: 1.3858 - val_sparse_categorical_accuracy: 0.2217
Epoch 3/10
1903/1903 [==============================] - 49s 26ms/step - loss: 1.3857 - sparse_categorical_accuracy: 0.2517 - val_loss: 1.3826 - val_sparse_categorical_accuracy: 0.2925
Epoch 4/10
1903/1903 [==============================] - 45s 24ms/step - loss: 1.3802 - sparse_categorical_accuracy: 0.3153 - val_loss: 1.3833 - val_sparse_categorical_accuracy: 0.2594
Epoch 5/10
1903/1903 [==============================] - 41s 22ms/step - loss: 1.3747 - sparse_categorical_accuracy: 0.3095 - val_loss: 1.3893 - val_sparse_categorical_accuracy: 0.3160
Epoch 6/10
1903/1903 [===========

In [277]:
RNN2_evaluate=model.evaluate(X_test,y_test)
print model.metrics_names,RNN2_evaluate

443/443 [==============================] - 8s 18ms/step
['loss', 'sparse_categorical_accuracy'] [1.354259140873601, 0.3792325057778886]


In [278]:
RNN_model2 = model

In [295]:
model = Sequential()
lstm = LSTM(units=10,use_bias=False,return_sequences=True,dropout=0.5)
# model.add(Dropout(0.5))
model.add(LSTM(units=10,use_bias=False,return_sequences=True,input_shape=(1000,22)))
model.add(lstm)
model.add(lstm)
model.add(lstm)
model.add(lstm)
# print model.output_shape
model.add(Dense(units=52,use_bias=True,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=4,use_bias=True,activation='softmax'))

In [296]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
history_RNN3=model.fit(X_train_valid,y_train_valid,epochs=10,batch_size=200,verbose=1,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/10
1903/1903 [==============================] - 56s 29ms/step - loss: 1.3869 - sparse_categorical_accuracy: 0.2459 - val_loss: 1.3859 - val_sparse_categorical_accuracy: 0.2689
Epoch 2/10
1903/1903 [==============================] - 40s 21ms/step - loss: 1.3862 - sparse_categorical_accuracy: 0.2480 - val_loss: 1.3861 - val_sparse_categorical_accuracy: 0.2547
Epoch 3/10
1903/1903 [==============================] - 40s 21ms/step - loss: 1.3851 - sparse_categorical_accuracy: 0.2780 - val_loss: 1.3852 - val_sparse_categorical_accuracy: 0.2783
Epoch 4/10
1903/1903 [==============================] - 39s 21ms/step - loss: 1.3827 - sparse_categorical_accuracy: 0.2769 - val_loss: 1.3840 - val_sparse_categorical_accuracy: 0.2877
Epoch 5/10
1903/1903 [==============================] - 39s 21ms/step - loss: 1.3807 - sparse_categorical_accuracy: 0.2743 - val_loss: 1.3811 - val_sparse_categorical_accuracy: 0.3160
Epoch 6/10
1903/1903 [===========

In [297]:
RNN3_evaluate=model.evaluate(X_test,y_test)
print model.metrics_names,RNN3_evaluate
RNN_model3=model

443/443 [==============================] - 9s 21ms/step
['loss', 'sparse_categorical_accuracy'] [1.1829827391536187, 0.5124153491471206]


In [298]:
model = Sequential()
lstm = LSTM(units=50,use_bias=False,return_sequences=True,dropout=0.5)
# model.add(Dropout(0.5))
model.add(LSTM(units=50,use_bias=False,return_sequences=True,input_shape=(1000,22)))
model.add(lstm)
model.add(lstm)
model.add(lstm)
model.add(lstm)
# print model.output_shape
model.add(Dense(units=52,use_bias=True,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=4,use_bias=True,activation='softmax'))

In [299]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
history_RNN4=model.fit(X_train_valid,y_train_valid,epochs=10,batch_size=200,verbose=1,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/10
1903/1903 [==============================] - 123s 65ms/step - loss: 1.3869 - sparse_categorical_accuracy: 0.2570 - val_loss: 1.3844 - val_sparse_categorical_accuracy: 0.2689
Epoch 2/10
1903/1903 [==============================] - 98s 52ms/step - loss: 1.3853 - sparse_categorical_accuracy: 0.2733 - val_loss: 1.3837 - val_sparse_categorical_accuracy: 0.2311
Epoch 3/10
1903/1903 [==============================] - 95s 50ms/step - loss: 1.3769 - sparse_categorical_accuracy: 0.3100 - val_loss: 1.3706 - val_sparse_categorical_accuracy: 0.4198
Epoch 4/10
1903/1903 [==============================] - 97s 51ms/step - loss: 1.3621 - sparse_categorical_accuracy: 0.3321 - val_loss: 1.3054 - val_sparse_categorical_accuracy: 0.4528
Epoch 5/10
1903/1903 [==============================] - 102s 54ms/step - loss: 1.2992 - sparse_categorical_accuracy: 0.4088 - val_loss: 1.2257 - val_sparse_categorical_accuracy: 0.4292
Epoch 6/10
1903/1903 [=========

In [301]:
RNN4_evaluate=model.evaluate(X_test,y_test)
print model.metrics_names,RNN4_evaluate
RNN_model4=model

443/443 [==============================] - 9s 20ms/step
['loss', 'sparse_categorical_accuracy'] [1.070020885015449, 0.5688487573886294]


In [302]:
model = Sequential()
lstm = LSTM(units=50,use_bias=False,return_sequences=True,dropout=0.5)
# model.add(Dropout(0.5))
model.add(LSTM(units=50,use_bias=False,return_sequences=True,input_shape=(1000,22)))
model.add(lstm)
model.add(lstm)
model.add(lstm)
model.add(lstm)
# print model.output_shape
model.add(Dense(units=52,use_bias=False,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=4,use_bias=False,activation='softmax'))

In [303]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
history_RNN5=model.fit(X_train_valid,y_train_valid,epochs=10,batch_size=200,verbose=1,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/10
1903/1903 [==============================] - 110s 58ms/step - loss: 1.3886 - sparse_categorical_accuracy: 0.2412 - val_loss: 1.3862 - val_sparse_categorical_accuracy: 0.2311
Epoch 2/10
1903/1903 [==============================] - 82s 43ms/step - loss: 1.3848 - sparse_categorical_accuracy: 0.2533 - val_loss: 1.3830 - val_sparse_categorical_accuracy: 0.3208
Epoch 3/10
1903/1903 [==============================] - 96s 50ms/step - loss: 1.3816 - sparse_categorical_accuracy: 0.2743 - val_loss: 1.3759 - val_sparse_categorical_accuracy: 0.3396
Epoch 4/10
1903/1903 [==============================] - 99s 52ms/step - loss: 1.3693 - sparse_categorical_accuracy: 0.3106 - val_loss: 1.3503 - val_sparse_categorical_accuracy: 0.3821
Epoch 5/10
1903/1903 [==============================] - 117s 62ms/step - loss: 1.3249 - sparse_categorical_accuracy: 0.3736 - val_loss: 1.2402 - val_sparse_categorical_accuracy: 0.4481
Epoch 6/10
1903/1903 [=========

In [304]:
RNN5_evaluate=model.evaluate(X_test,y_test)
print model.metrics_names,RNN5_evaluate
RNN_model5=model

443/443 [==============================] - 9s 20ms/step
['loss', 'sparse_categorical_accuracy'] [1.0206193251211542, 0.5598194129580031]


In [305]:
model = Sequential()
lstm = LSTM(units=100,use_bias=False,return_sequences=True,dropout=0.5)
# model.add(Dropout(0.5))
model.add(LSTM(units=100,use_bias=False,return_sequences=True,input_shape=(1000,22)))
model.add(lstm)
model.add(lstm)
model.add(lstm)
model.add(lstm)
# print model.output_shape
model.add(Dense(units=100,use_bias=True,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=4,use_bias=True,activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
history_RNN6=model.fit(X_train_valid,y_train_valid,epochs=20,batch_size=200,verbose=1,validation_split = 0.1)

Train on 1903 samples, validate on 212 samples
Epoch 1/20
1903/1903 [==============================] - 324s 170ms/step - loss: 1.3883 - sparse_categorical_accuracy: 0.2638 - val_loss: 1.3854 - val_sparse_categorical_accuracy: 0.2689
Epoch 2/20
1903/1903 [==============================] - 303s 159ms/step - loss: 1.3842 - sparse_categorical_accuracy: 0.2643 - val_loss: 1.3810 - val_sparse_categorical_accuracy: 0.3538
Epoch 3/20
1903/1903 [==============================] - 299s 157ms/step - loss: 1.3637 - sparse_categorical_accuracy: 0.3379 - val_loss: 1.3061 - val_sparse_categorical_accuracy: 0.4009
Epoch 4/20
1903/1903 [==============================] - 359s 189ms/step - loss: 1.2576 - sparse_categorical_accuracy: 0.4430 - val_loss: 1.1351 - val_sparse_categorical_accuracy: 0.4811
Epoch 5/20
1903/1903 [==============================] - 346s 182ms/step - loss: 1.1490 - sparse_categorical_accuracy: 0.5102 - val_loss: 1.0310 - val_sparse_categorical_accuracy: 0.5849
Epoch 6/20
1903/1903 [=

In [ ]:
RNN6_evaluate=model.evaluate(X_test,y_test)
print model.metrics_names,RNN6_evaluate
RNN_model6=model